# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

<b>Цель</b>-проанализировать базу данных,сформулировать выводы.

# Содержание

<ul>
    <li>Cтруктура данных</li>
    <li>Загрузка данных</li>
    <li>SQL-запросы для решения каждого задания</li>
    <li>Вывод</li>
</ul>

# Cтруктура данных

<ul>Таблица books
<li>book_id — идентификатор книги;</li>
<li>author_id — идентификатор автора</li>
<li>title — название книги;</li>
<li>num_pages — количество страниц;</li>
<li>publication_date — дата публикации книги;</li>
<li>publisher_id — идентификатор издателя.</li>
</ul>

<ul>Таблица authors
<li>author_id — идентификатор автора;</li>
<li>author — имя автора.</li>
</ul>

<ul>Таблица authors
<li>publisher_id — идентификатор издательства;</li>
<li>publisher — название издательства;</li>
</ul>

<ul>Таблица ratings
<li>rating_id — идентификатор оценки;</li>
<li>book_id — идентификатор книги;</li>
<li>username — имя пользователя, оставившего оценку;</li>
<li>rating — оценка книги.</li>
</ul>

<ul>Таблица reviews
<li>review_id — идентификатор обзора;</li>
<li>book_id — идентификатор книги;</li>
<li>username — имя автора обзора;</li>
<li>text — текст обзора.</li>
</ul>


# Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

In [2]:
# Функция для вывода результатов запросов
def sql_result(query):
  return pd.io.sql.read_sql(query, con = engine)

Вывели 5 строк таблицы books

In [3]:
query = '''SELECT * FROM books LIMIT 5'''
sql_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Вывели 5 строк таблицы authors

In [4]:
query = '''SELECT * FROM authors LIMIT 5'''
sql_result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Вывели 5 строк таблицы publishers

In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''
sql_result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Вывели 5 строк таблицы ratings

In [6]:
query = '''SELECT * FROM ratings LIMIT 5'''
sql_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Вывели 5 строк таблицы reviews

In [7]:
query = '''SELECT * FROM reviews LIMIT 5'''
sql_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# SQL-запросы для решения каждого задания

## Cколько книг вышло после 1 января 2000 года

In [8]:
query=''' 
    SELECT COUNT(book_id)
    FROM books
    WHERE CAST(publication_date AS date)>'2000-01-01'
'''
sql_result(query)

,count
0,819


## Для каждой книги посчитали количество обзоров и среднюю оценку;

In [9]:
query='''
    WITH
    rat_books AS ( SELECT book_id,
                          AVG(rating) AS avg_rating
                  FROM ratings
                  GROUP BY book_id),
    rev_count AS ( SELECT book_id,
                          COUNT(review_id) AS count_reviews
                   FROM reviews
                   GROUP BY book_id)
                  
    SELECT title,
           count_reviews,
           avg_rating
           
    FROM books JOIN rev_count ON books.book_id=rev_count.book_id
    JOIN rat_books ON books.book_id=rat_books.book_id
    ORDER BY count_reviews DESC
'''
sql_result(query)

,title,count_reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Road,6,3.772727
2,The Book Thief,6,4.264151
3,The Glass Castle,6,4.206897
4,Water for Elephants,6,3.977273
...,...,...,...
989,Naked Empire (Sword of Truth #8),1,3.500000
990,Moo Baa La La La!,1,3.000000
991,Merrick (The Vampire Chronicles #7),1,4.000000
992,Babyville,1,3.500000


## Определили издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
query='''
       WITH
       id_pub AS (SELECT publisher_id,
                    COUNT(book_id) AS count_book
            FROM books
            WHERE num_pages>50
            GROUP BY publisher_id
            ORDER BY count_book DESC
            LIMIT 1)
        SELECT publisher,
               id_pub.count_book
        FROM publishers JOIN id_pub ON publishers.publisher_id=id_pub.publisher_id     
'''
sql_result(query)

,publisher,count_book
0,Penguin Books,42


## Определили автора с самой высокой средней оценкой книг — учитывали только книги с 50 и более оценками

In [11]:
query='''  WITH 
        b_id AS (SELECT book_id,
                        COUNT(rating_id),
                        AVG(rating) AS a_rat
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id)>=50
                    ORDER BY AVG(rating) DESC),
        aut_id AS ( SELECT author_id,
                           b_id.a_rat AS rat
                    FROM books
                    JOIN b_id ON books.book_id=b_id.book_id
        ),
        id AS  (SELECT author_id,
                        AVG(rat) AS avg_rat
                FROM aut_id
                GROUP BY author_id
                ORDER BY AVG(rat) DESC
                LIMIT 1)
        SELECT author,
               id.avg_rat
        FROM authors JOIN id ON authors.author_id=id.author_id
       '''
sql_result(query)

,author,avg_rat
0,J.K. Rowling/Mary GrandPré,4.283844


## Посчитаи среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query='''
        WITH 
            
       name AS  (SELECT username,
                    COUNT(rating_id)
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id)>48
            ORDER BY COUNT(rating_id)), 
        count_rev AS ( SELECT reviews.username,
                           COUNT (review_id) AS c_r
                    FROM reviews JOIN name ON reviews.username=name.username
                    GROUP BY reviews.username)
        SELECT AVG(c_r) AS avg_count
        FROM count_rev
    '''
sql_result(query)

,avg_count
0,24.0


# Выводы

<ul>
    <li>После 1 января 2000 года вышло 819</li>
    <li>Книга Twilight (Twilight №1) получила больше всех обзоров-7,средняя оценка 3.6</li>
    <li>Издательство Penguin Books  выпустило наибольшее число книг(42) толще 50 страниц</li>
    <li>J.K. Rowling/Mary GrandPré-автор с самой высокой средней оценкой книг-4.28</li>
    <li>Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок-24</li>
</ul>